In [101]:
# make a nn using flax and jax to train on a simple XOR function
import jax
import jax.numpy as jnp
from flax import linen as nn
from jax import random
import optax

In [102]:
devices = jax.devices()
print(devices)

[cuda(id=0)]


In [103]:
import jax
import jax.numpy as jnp
import flax.linen as nn
import optax

In [104]:
class SimpleNN(nn.Module):
    hidden_size: int

    @nn.compact
    def __call__(self, x):
        x = nn.Dense(self.hidden_size)(x)
        x = nn.relu(x)
        x = nn.Dense(1)(x)
        return x
    
    
@jax.jit
def train_step(params, opt_state, x, y):
    grads = jax.grad(mse_loss)(params, model, x, y)
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    return params, opt_state

def mse_loss(params, model, x, y):
    preds = model.apply(params, x)
    loss = jnp.mean((preds - y) ** 2)
    return loss


In [105]:
# Hyperparameters
hidden_size = 10
input_size = 10
# x = jnp.ones((1, input_size))  # Dummy input, rows, columns

x = jnp.arange(0, input_size + 1)
y = x[::-1]

#create XOR y and x data
#x = jnp.array([[0, 0], [0, 1], [1, 0], [1, 1]])
#y = jnp.array([0, 1, 1, 0])


In [106]:

# Create the model
model = SimpleNN(hidden_size=hidden_size)

# Initialize parameters
rng = jax.random.PRNGKey(2)

params = model.init(rng, x)

In [107]:
# Create an optimizer
optimizer = optax.adam(learning_rate=0.01)
opt_state = optimizer.init(params)
# Perform a single training step
params, opt_state = train_step(params, opt_state, x, y)

# Training loop
num_epochs = 50000

for epoch in range(num_epochs):
    params, opt_state = train_step(params, opt_state, x, y)
    if epoch % 10 == 0:
        loss = mse_loss(params, model, x, y)
        print(f'Epoch {epoch}, Loss: {loss:.4f}')



Epoch 0, Loss: 19.6402
Epoch 10, Loss: 13.2857
Epoch 20, Loss: 10.6609
Epoch 30, Loss: 10.0651
Epoch 40, Loss: 10.1602
Epoch 50, Loss: 10.0339
Epoch 60, Loss: 10.0000
Epoch 70, Loss: 10.0035
Epoch 80, Loss: 10.0028
Epoch 90, Loss: 10.0009
Epoch 100, Loss: 10.0002
Epoch 110, Loss: 10.0000
Epoch 120, Loss: 10.0000
Epoch 130, Loss: 10.0000
Epoch 140, Loss: 10.0000
Epoch 150, Loss: 10.0000
Epoch 160, Loss: 10.0000
Epoch 170, Loss: 10.0000
Epoch 180, Loss: 10.0000
Epoch 190, Loss: 10.0000
Epoch 200, Loss: 10.0000
Epoch 210, Loss: 10.0000
Epoch 220, Loss: 10.0000
Epoch 230, Loss: 10.0000
Epoch 240, Loss: 10.0000
Epoch 250, Loss: 10.0000
Epoch 260, Loss: 10.0000
Epoch 270, Loss: 10.0000
Epoch 280, Loss: 10.0000
Epoch 290, Loss: 10.0000
Epoch 300, Loss: 10.0000
Epoch 310, Loss: 10.0000
Epoch 320, Loss: 10.0000
Epoch 330, Loss: 10.0000
Epoch 340, Loss: 10.0000
Epoch 350, Loss: 10.0000
Epoch 360, Loss: 10.0000
Epoch 370, Loss: 10.0000
Epoch 380, Loss: 10.0000
Epoch 390, Loss: 10.0000
Epoch 400, 